Imaging methods such as MRI and CT result in 3-D volumetric representations of the data, e.g. with 256x256x256 voxels. You can think of this representation as having a “voxel” coordinate system, where voxel (1, 1, 1) is the first and (256, 256, 256) the last in the volume. The voxel coordinate system however does not specify the physical dimensions (e.g. mm or cm) and does not specify how the head (which is somewhere within the volume) relates to the voxel indices. Therefore a volumetric description of imaging data as a 3-D array has to be complemented with a description of the head coordinate system. This description is commonly implemented using a 4×4 homogenous coordinate transformation matrix, for which an excellent description is available here.

The origin of the MNI coordinate system is the anterior commissure
The X-axis extends from the left side of the brain to the right side
The Y-axis points from posterior to anterior
The Z-axis points from inferior to superior

MNI or SPM is used if the geometry is spatially warped to the MNI152 template brain.

A Nifti (Neuroimaging Informatics Technology Initiative) image contains, along with its 3D or 4D data content, a 4x4 matrix
encoding an affine transformation that maps the data array into millimeter
space. If (i, j, k) encodes an integer position (voxel) with the data array,
then adding 1 as a fourth entry, (i, j, k, 1), and multiplying by the affine
matrix yields (x, y, z, 1), a 4-vector containing the millimeter position of
the voxel.

MNI units:	mm	
scaled to match averaged template
- orientation: RAS
- origin: anterior commissure 
- scaling: scaled to match averaged template

MNI 152 is a reference map. It's an average brain taken from MRI's of 152 healthy individuals, normalized based on MNI 305, a former iteration based on 305 MRI scans of healthy individuals (male and female), which used as reference previous maps produced by hand.

MNI means "Montreal Neurosciences Institute". So, the MNI 152 is the averaging of 152 3DT1 coming from 152 different peoples. It means that when you visualize the MNI152 image, you see an "average brain".

 Normalization is the process of warping a brain to match a standard size, orientation and shape of other brains.

Din introducerea la interactive selection: The starting point of these experiments  was the lack of MNI-coordinate ranges displayed
for each slice in the `nilearn` interactive plotting, as well as the previously unknown point coordinates of a clicked point. With Plotly we can remedy these drawbacks  to give a more informative slice view.

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [ ]:
import plotly.graph_objs as go

In [ ]:
import numpy as np
import copy
from matplotlib import cm as mpl_cm
import matplotlib as mpl
from nilearn import (plotting, _utils)

In [ ]:
from nilearn import datasets

# haxby dataset to have EPI images and masks
haxby_dataset = datasets.fetch_haxby()

# print basic information on the dataset
"""
print('First subject anatomical nifti image (3D) is at: %s' %
      haxby_dataset.anat[0])
print('First subject functional nifti image (4D) is at: %s' %
      haxby_dataset.func[0])  # 4D data
"""
haxby_anat_filename = haxby_dataset.anat[0]
haxby_mask_filename = haxby_dataset.mask_vt[0]
haxby_func_filename = haxby_dataset.func[0]

# one motor contrast map from NeuroVault
motor_images = datasets.fetch_neurovault_motor_task()
stat_img = motor_images.images[0]


In [ ]:
print(stat_img)

In [ ]:
def mpl_to_plotly(cmap, pl_entries):
    h=1.0/(pl_entries-1)
    pl_colorscale=[]
    for k in range(pl_entries):
        C=list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([round(k*h,2), f'rgb({C[0]}, {C[1]}, {C[2]})'])
    return pl_colorscale

In [ ]:
def colorscale(cmap, values, threshold=None, symmetric_cmap=True,
               vmax=None, vmin=None):
    """this function modifies nilearn.plotting.js_plotting_utils.colorscale
    It defines a Plotly colorscale from a given nilearn or matplotlib colormap,
    extracts the color range and the threshold"""
    
    cmap = mpl_cm.get_cmap(cmap)
    abs_values = np.abs(values)
    
    if not symmetric_cmap and (values.min() < 0):
        warnings.warn('you have specified symmetric_cmap=False '
                      'but the map contains negative values; '
                      'setting symmetric_cmap to True')
        symmetric_cmap = True
    if symmetric_cmap and vmin is not None:
        warnings.warn('vmin cannot be chosen when cmap is symmetric')
        vmin = None
    if threshold is not None:
        if vmin is not None:
            warnings.warn('choosing both vmin and a threshold is not allowed; '
                          'setting vmin to 0')
        vmin = 0
    if vmax is None:
        vmax = abs_values.max()
    if symmetric_cmap:
        vmin = - vmax
    if vmin is None:
        vmin = values.min()
    
    
   
    abs_threshold = None
    if threshold is not None:
        abs_threshold = _utils.param_validation.check_threshold(threshold, values, _utils.extmath.fast_abs_percentile)
        norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
        ca = norm(-abs_threshold)
        cb = norm(abs_threshold)
        h1 = ca/11
        dl = [k*h1 for k in range(11)]
        h2 = (cb-ca) / 10
        dc = [ca+k*h2 for k in range(11)]
        h3 = (1-cb-h2) / 10
        dr = [cb+h2+k*h3 for k in range(11)]
        d = dl+dc+dr
        cmaplist = [cmap(t)[:3] for t in d]
        for k in range(11):  
            cmaplist[k+11] = mpl_cm.gray(k*0.1)[:3]
            
        pl_colorscale = []
   
        for k, t in enumerate(d):
            c = list(map(np.uint8, np.array(cmaplist[k])*255))
            pl_colorscale.append([round(t,3), f'rgb({c[0]}, {c[1]}, {c[2]})'])  
    else:
        pl_colorscale = mpl_to_plotly(cmap, 11)
    return  {
        'colorscale': pl_colorscale, 'vmin': vmin, 'vmax': vmax, 
        'abs_threshold': abs_threshold}

In [ ]:
def pl_view_img(stat_map_img,  bg_img='MNI152', cut_coords= None,
             threshold=1e-6,
             cmap=plotting.cm.cold_hot,
             symmetric_cmap=True,
             dim='auto',
             vmax=None,
             vmin=None,
             resampling_interpolation='continuous',
             **kwargs):
    """
    Interactive Plotly viewer of a statistical map
    ----------
    stat_map_img : Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        The statistical map image. Can be either a 3D volume or a 4D volume
        with exactly one time point.
    bg_img : Niimg-like object (default='MNI152')
        See http://nilearn.github.io/manipulating_images/input_output.html
        The background image that the stat map will be plotted on top of.
        If nothing is specified, the MNI152 template will be used.
        To turn off background image, just pass "bg_img=False".
    cut_coords : None, or a tuple of floats (default None)
        The MNI coordinates of the point where the cut is performed
        as a 3-tuple: (x, y, z). If None is given, the cuts are calculated
        automaticaly.
    threshold : string, number or None  (default=1e-6)
        If None is given, the image is not thresholded.
        If a string of the form "90%" is given, use the 90-th percentile of
        the absolute value in the image.
        If a number is given, it is used to threshold the image:
        values below the threshold (in absolute value) are plotted
        as transparent. If auto is given, the threshold is determined
        automatically.
    
    cmap : matplotlib colormap, optional
        The colormap for specified image.
    symmetric_cmap : bool, optional (default=True)
        True: make a symmetric colorscale   (ranging from -vmax to vmax).
        False: the colormap will go from the minimum of the volume to vmax.
        Set it to False if you are plotting a positive volume, e.g. an atlas
        or an anatomical image.
    dim : float, 'auto' (default='auto')
        Dimming factor applied to background image. By default, automatic
        heuristics are applied based upon the background image intensity.
        Accepted float values, where a typical scan is between -2 and 2
        (-2 = increase constrast; 2 = decrease contrast), but larger values
        can be used for a more pronounced effect. 0 means no dimming.
    vmax : float, or None (default=None)
        max value for mapping colors.
        If vmax is None and symmetric_cmap is True, vmax is the max
        absolute value of the volume.
        If vmax is None and symmetric_cmap is False, vmax is the max
        value of the volume.
    vmin : float, or None (default=None)
        min value for mapping colors.
        If `symmetric_cmap` is `True`, `vmin` is always equal to `-vmax` and
        cannot be chosen.
        If `symmetric_cmap` is `False`, `vmin` defaults to the min of the
        image, or 0 when a threshold is used.
    resampling_interpolation : string, optional (default continuous)
        The interpolation method for resampling.
        Can be 'continuous', 'linear', or 'nearest'.
        See nilearn.image.resample_img
    
    Returns
    -------
     a dict with keys: plotly colorscale, vmin, vmax, abs_threshold
        
    """    
   
    mask_img, stat_map_img, data, threshold = plotting.html_stat_map._mask_stat_map(stat_map_img, threshold)
    color_info = colorscale(cmap, data.ravel(), threshold=threshold, 
                        symmetric_cmap=symmetric_cmap, vmax=vmax,
                        vmin=vmin)
  
    bg_img, bg_min, bg_max, black_bg = plotting.html_stat_map._load_bg_img(stat_map_img, bg_img, dim)
    
    stat_map_img, mask_img = plotting.html_stat_map._resample_stat_map(stat_map_img, bg_img, mask_img,
                                                resampling_interpolation)
   
    A = stat_map_img.affine
    cut_slices =  plotting.html_stat_map._get_cut_slices(stat_map_img, cut_coords, threshold)
    
    return [color_info, _utils.niimg._safe_get_data(bg_img, ensure_finite=True),  
            _utils.niimg._safe_get_data(stat_map_img, ensure_finite=True), 
            _utils.niimg._safe_get_data(mask_img, ensure_finite=True), cut_slices, A]


Extract data from the NIfTI images bg_img, mask_img, stat_map_img, define the plotly colorscale:

In [ ]:
#stat_img = 'image_10426.nii.gz'
cut_coords=[13, -27, -20]#z was 50  x=13
color_info, bg_img, stat_map_img, mask_img, cut_slices, A = pl_view_img(stat_img, cut_coords= cut_coords, threshold=3)

min-max MNI-coordinates  corresponding to min-max voxel indices i, j, k:

In [ ]:
xMNI_min, yMNI_min, zMNI_min = A[:, 3][:3]
imax, jmax, kmax =stat_map_img.shape 
xMNI_max, yMNI_max, zMNI_max, one = np.dot(A, [imax-1, jmax-1, kmax-1, 1])

In [ ]:
pl_colorscale=color_info['colorscale']
vmin=color_info['vmin']
vmax=color_info['vmax']
abs_threshold=color_info['abs_threshold']
islice, jslice, kslice = np.array(cut_slices-1, int)# voxel indices corresponding to cut_slices

Mix the backgraound image and statistical image values according to mask_img:

In [ ]:
a, b = -abs_threshold, abs_threshold
vmin_bg, vmax_bg = bg_img.min(), bg_img.max()
new_stat_img = copy.deepcopy(stat_map_img)
alpha = (b-a) / (vmax_bg-vmin_bg)  
new_bg = a + alpha * (bg_img-vmin_bg) # map bg_img vals to [a,b]
I, J, K = np.where(mask_img==1)
new_stat_img[I, J, K] = new_bg[I, J, K]

Define the 2d arrays cooresponding to the three slices:

In [ ]:
xsts = new_stat_img[islice, :, :]
ysts = new_stat_img[:, jslice, :]
zsts = new_stat_img[:, :, kslice]

### Plotly plots of  slices in the 3d space referenced to the MNI-system of coordinates

Define the arrays containing the voxel coordinates of the points in each slice:

In [ ]:
yx = list(range(xsts.shape[0]))
zx = list(range(xsts.shape[1]))
yx, zx = np.meshgrid(yx, zx)
xx = islice * np.ones(xsts.T.shape)

xy = list(range(ysts.shape[0]))
zy = list(range(ysts.shape[1]))
xy, zy = np.meshgrid(xy, zy)
yy = jslice * np.ones(ysts.T.shape)

xz = list(range(zsts.shape[0]))
yz = list(range(zsts.shape[1]))
xz, yz = np.meshgrid(xz,yz)
zz = kslice * np.ones(zsts.T.shape)

Transform voxel indices to MNI coordinates:

In [ ]:
xx, yx, zx, one_arr = np.einsum('ik, kjm -> ijm', A, np.stack((xx, yx, zx, np.ones(xx.shape)))) 
xy, yy, zy, one_arr = np.einsum('ik, kjm -> ijm', A, np.stack((xy, yy, zy, np.ones(yy.shape)))) 
xz, yz, zz, one_arr = np.einsum('ik, kjm -> ijm', A, np.stack((xz, yz, zz, np.ones(zz.shape)))) 

Define the Plotly surfaces representing the three slices:

In [ ]:
slicex=dict(type='surface',
            x=xx,
            y=yx,
            z=zx,
            name='x-slice',
            surfacecolor=xsts.T,
            colorscale=pl_colorscale,     
            colorbar=dict(thickness=20, ticklen=4, tick0=-7, dtick=2, len=0.75),
            cmin=vmin,
            cmax=vmax
           )

slicey=dict(type='surface',
            x=xy,
            y=yy,
            z=zy,
            name='y-slice',
            surfacecolor=ysts.T,
            colorscale=pl_colorscale,    
            colorbar=dict(thickness=20, ticklen=4, tick0=-7, dtick=2, len=0.75),
            cmin=vmin,
            cmax=vmax)
slicez=dict(type='surface',
            x=xz,
            y=yz,
            z=zz,
            name='z-slice',
            surfacecolor=zsts.T,
            colorscale=pl_colorscale, 
            colorbar=dict(thickness=20, ticklen=4, tick0=-7, dtick=2, len=0.75),
            cmin=vmin,
            cmax=vmax)                 

In [ ]:
axis3d = dict(showbackground=True, 
            backgroundcolor="rgb(230, 230,230)",
            gridcolor="rgb(255, 255, 255)",      
            zerolinecolor="rgb(255, 255, 255)", 
            ticklen=4,
            tickfont=dict(size=11)  
            )


layout = dict(width=700,
              height=700,
              scene=dict(xaxis=dict(axis3d),
                    yaxis=dict(axis3d), 
                    zaxis=dict(axis3d),
                    camera=dict(eye=dict(x=-1.25, y=-1.25, z=1))     
                    ),
         )

In [ ]:
figxyz=go.Figure(data=[slicex, slicey, slicez], layout=layout)
figxyz['layout']['title']=f'Orthogonal brain slices through the point<br> of  MNI coords {cut_coords}'

iplot(figxyz)

In [ ]:
import plotly.plotly as py
#py.sign_in('empet', 'isVGpT2zbkjE5AzU5pGr')
#py.iplot(figxyz, filename='ortho-brain-slices')

In [ ]:
figx = go.Figure(data=[slicex], layout=layout)
figx.layout.title = 'A statistical map mapped on a  brain x-slice background'
figx.layout.scene.xaxis.range=[xMNI_min, xMNI_max]
#iplot(figx)

In [ ]:
import plotly.plotly as py
#py.sign_in('empet', 'isVGpT2zbkjE5AzU5pGr')
#py.iplot(figx, filename='brain-x-slice')

In [ ]:
figz=go.Figure(data=[slicez], layout=layout)
figz['layout']['title']='Brain z-slice'
figz.layout.scene.zaxis.range=[zMNI_min, zMNI_max]
#iplot(figz)

In [ ]:
figy=go.Figure(data=[slicey], layout=layout)
figy['layout']['title']='Brain y-slice'
figy.layout.scene.yaxis.range=[yMNI_min, yMNI_max]
#iplot(figy)

## 2d plots of slices

In [ ]:
import plotly.graph_objs as go
import ipywidgets as ipw

In [ ]:
xx2 = np.linspace(yMNI_min, yMNI_max, jmax)
yx2 = np.linspace(zMNI_min, zMNI_max, kmax)
textx2 = [[f'y: {xx2[j]}<br>z: {yx2[i]}<br>val:'+'{:.2f}'.format(xsts.T[i,j]) for j in range(jmax)] 
          for i in range(kmax)]

slicex2 = dict(type='heatmap',
               #zsmooth='best',
               x=xx2,
               y=yx2,
               z=xsts.T,
               text=textx2, 
               colorscale=pl_colorscale,     
               colorbar=dict(thickness=20, ticklen=4, tick0=-7, dtick=2),
               showscale=False, 
               hoverinfo='text', 
               zmin=vmin,
               zmax=vmax)

xy2 = np.linspace(xMNI_min, xMNI_max, imax)
yy2 = np.linspace(zMNI_min, zMNI_max, kmax)
texty2 = [[f'x: {xy2[j]}<br>z: {yy2[i]}<br>val:'+'{:.2f}'.format(ysts.T[i,j]) for j in range(imax)]
          for i in range(kmax)]

slicey2 = dict(type='heatmap',
               x=xy2,
               y=yy2,  
               z=ysts.T,
               text=texty2, 
               #zsmooth='best', 
               colorscale=pl_colorscale,    
               colorbar=dict(thickness=20, ticklen=4, tick0=-7, dtick=2),
               showscale=False, 
               hoverinfo='text', 
               zmin=vmin,
               zmax=vmax)

xz2 = np.linspace(xMNI_min, xMNI_max, imax)
yz2 = np.linspace(yMNI_min, yMNI_max, jmax)
textz2 = [[f'x: {xz2[j]}<br>y: {yz2[i]}<br>val:'+'{:.2f}'.format(zsts.T[i,j]) for j in range(imax)]
          for i in range(jmax)]

slicez2 = dict(type='heatmap',
               #zsmooth='best',
               x=xz2,
               y=yz2, 
               z=zsts.T,
               text=textz2, 
               colorscale=pl_colorscale, 
               colorbar=dict(thickness=20, ticklen=4, tick0=-7, dtick=2),
               hoverinfo='text',
               zmin=vmin,
               zmax=vmax)
                   

In [ ]:
axis2d = dict(ticklen=4, showticklabels=True)
layout2d = dict(width=400, height=400,
                xaxis=axis2d,
                yaxis=axis2d)
   

In [ ]:
fig2dx = go.FigureWidget(data=[slicex2], layout=layout2d)
with fig2dx.batch_update():
    fig2dx.layout.title = f'Slice x={cut_coords[0]}'
    fig2dx.layout.xaxis.update(range=[yMNI_min, yMNI_max], title='y')
    fig2dx.layout.yaxis.update(range=[zMNI_min, zMNI_max], title='z')
#fig2dx  

In [ ]:
fig2dy = go.FigureWidget(data=[slicey2], layout=layout2d)
with fig2dy.batch_update():
    fig2dy.layout.title=f'Slice y={cut_coords[1]}'
    fig2dy.layout.xaxis.update(range=[xMNI_min, xMNI_max], title='x')
    fig2dy.layout.yaxis.update(range=[zMNI_min, zMNI_max], title='z')

In [ ]:
fig2dz = go.FigureWidget(data=[slicez2], layout=layout2d)
with fig2dz.batch_update():
    fig2dz.layout.height=425
    fig2dz.layout.title=f'Slice z={cut_coords[2]}'
    fig2dz.layout.xaxis.update(range=[xMNI_min, xMNI_max], title='x')
    fig2dz.layout.yaxis.update(range=[yMNI_min, yMNI_max], title='y')
#fig2dz 

In [ ]:
ipw.HBox([fig2dx, fig2dy, fig2dz])

Compare to nilearn view_img:

In [ ]:
%matplotlib inline

view = plotting.view_img(stat_img, cut_coords=(13, -27, 50), threshold=3)
view

In [ ]:
plotting.plot_stat_map(stat_img,
                       threshold=3, title="plot_stat_map",
                       cut_coords=(13, -27, 50))
